In [1]:
import base64
import collections
import datetime
import itertools
import json
import os
import pickle
import random
import re
import sys
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
#import PIL
import pylab
import scipy
#import seaborn as sns
import sklearn
from sklearn import *
import statsmodels as sm
import xgboost as xgb

np.random.seed(1337)

%matplotlib inline

#sns.set(font_scale=1.0)
mpl.rcParams['figure.figsize'] = 10, 6
#sns.set_style('whitegrid')
#sns.set_palette(sns.color_palette('muted'))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/

In [2]:
from urllib import urlopen
from bs4 import BeautifulSoup

# Processing Raw Text

## Accessing Text from the Web and from Disk

### Electronic Books

In [3]:
url = "http://www.gutenberg.org/files/2554/2554.txt"
raw = urlopen(url).read()
print type(raw)
print len(raw)
raw[:75]

<type 'str'>
1176896


'The Project Gutenberg EBook of Crime and Punishment, by Fyodor Dostoevsky\r\n'

In [4]:
tokens = nltk.word_tokenize(raw)
print type(tokens)
print len(tokens)
print tokens[:10]

<type 'list'>
254352
['The', 'Project', 'Gutenberg', 'EBook', 'of', 'Crime', 'and', 'Punishment', ',', 'by']


In [5]:
text = nltk.Text(tokens)
print type(text)
print text[1020:1060]
print
print text.collocations()

<class 'nltk.text.Text'>
['AND', 'PUNISHMENT', 'PART', 'I', 'CHAPTER', 'I', 'On', 'an', 'exceptionally', 'hot', 'evening', 'early', 'in', 'July', 'a', 'young', 'man', 'came', 'out', 'of', 'the', 'garret', 'in', 'which', 'he', 'lodged', 'in', 'S.', 'Place', 'and', 'walked', 'slowly', ',', 'as', 'though', 'in', 'hesitation', ',', 'towards', 'K.']

Katerina Ivanovna; Pyotr Petrovitch; Pulcheria Alexandrovna; Avdotya
Romanovna; Rodion Romanovitch; Marfa Petrovna; Sofya Semyonovna; old
woman; Project Gutenberg-tm; Porfiry Petrovitch; Amalia Ivanovna;
great deal; Nikodim Fomitch; young man; Ilya Petrovitch; n't know;
Project Gutenberg; Dmitri Prokofitch; Andrey Semyonovitch; Hay Market
None


In [6]:
raw.find("PART I")

5338

In [7]:
raw.rfind("End of Project Gutenberg's Crime")

1157746

### Dealing with HTML

In [8]:
url = "http://news.bbc.co.uk/2/hi/health/2284783.stm"
html = urlopen(url).read()
print html[:60]

<!doctype html public "-//W3C//DTD HTML 4.0 Transitional//EN


In [9]:
soup = BeautifulSoup(html, 'html.parser')
raw = soup.get_text()
tokens = nltk.word_tokenize(raw)
print tokens[:10]

[u'BBC', u'NEWS', u'|', u'Health', u'|', u'Blondes', u"'to", u'die', u'out', u'in']


In [10]:
tokens = tokens[96:399]
text = nltk.Text(tokens)
text.concordance('gene')

Displaying 5 of 5 matches:
hey say too few people now carry the gene for blondes to last beyond the next 
blonde hair is caused by a recessive gene . In order for a child to have blond
 have blonde hair , it must have the gene on both sides of the family in the g
ere is a disadvantage of having that gene or by chance . They do n't disappear
des would disappear is if having the gene was a disadvantage and I do not thin


### Reading Local Files

In [11]:
f = open('data/the_old_man_and_the_sea.txt')
raw = f.read()

### The NLP Pipeline

In [12]:
f = open('data/the_old_man_and_the_sea.txt')
raw = f.read()
type(raw)

str

In [13]:
tokens = nltk.word_tokenize(raw)
type(tokens)

list

In [14]:
words = [w.lower() for w in tokens]
type(words)

list

In [15]:
vocab = sorted(set(words))
type(vocab)

list

## Regular Expressions for Detecting Word Patterns

### Using Basic Metacharacters

In [17]:
wordlist = [w for w in nltk.corpus.words.words('en') if w.islower()]
[w for w in wordlist if re.search('ed$', w)][:10]

[u'abaissed',
 u'abandoned',
 u'abased',
 u'abashed',
 u'abatised',
 u'abed',
 u'aborted',
 u'abridged',
 u'abscessed',
 u'absconded']

In [18]:
[w for w in wordlist if re.search('^..j..t..$', w)][:10]

[u'abjectly',
 u'adjuster',
 u'dejected',
 u'dejectly',
 u'injector',
 u'majestic',
 u'objectee',
 u'objector',
 u'rejecter',
 u'rejector']

### Ranges and Closures

In [19]:
[w for w in wordlist if re.search('^[ghi][mno][jlk][def]$', w)]

[u'gold', u'golf', u'hold', u'hole']

In [20]:
chat_words = sorted(set(w for w in nltk.corpus.nps_chat.words()))
[w for w in chat_words if re.search('^m+i+n+e+$', w)]

[u'miiiiiiiiiiiiinnnnnnnnnnneeeeeeeeee',
 u'miiiiiinnnnnnnnnneeeeeeee',
 u'mine',
 u'mmmmmmmmiiiiiiiiinnnnnnnnneeeeeeee']

In [22]:
[w for w in chat_words if re.search('^[ha]+$', w)][:10]

[u'a',
 u'aaaaaaaaaaaaaaaaa',
 u'aaahhhh',
 u'ah',
 u'ahah',
 u'ahahah',
 u'ahh',
 u'ahhahahaha',
 u'ahhh',
 u'ahhhh']

In [24]:
wsj = sorted(set(nltk.corpus.treebank.words()))
[w for w in wsj if re.search('^[0-9]+\.[0-9]+$', w)][:10]

[u'0.0085',
 u'0.05',
 u'0.1',
 u'0.16',
 u'0.2',
 u'0.25',
 u'0.28',
 u'0.3',
 u'0.4',
 u'0.5']

In [25]:
[w for w in wsj if re.search('^[A-Z]+\$$', w)][:10]

[u'C$', u'US$']

In [26]:
[w for w in wsj if re.search('^[0-9]{4}$', w)][:10]

[u'1614',
 u'1637',
 u'1787',
 u'1901',
 u'1903',
 u'1917',
 u'1925',
 u'1929',
 u'1933',
 u'1934']

In [27]:
[w for w in wsj if re.search('^[a-z]{5,}-[a-z]{2,3}-[a-z]{,6}$', w)][:10]

[u'black-and-white',
 u'bread-and-butter',
 u'father-in-law',
 u'machine-gun-toting',
 u'savings-and-loan']

## Useful Applications of Regular Expressions

### Doing More with Word Pieces

In [28]:
rotokas_words = nltk.corpus.toolbox.words('rotokas.dic')
cvs = [cv for w in rotokas_words for cv in re.findall(r'[ptksvr][aeiou]', w)]
cfd = nltk.ConditionalFreqDist(cvs)
cfd.tabulate()

    a   e   i   o   u 
k 418 148  94 420 173 
p  83  31 105  34  51 
r 187  63  84  89  79 
s   0   0 100   2   1 
t  47   8   0 148  37 
v  93  27 105  48  49 


In [30]:
# Examining the rows for s and t, we see they are in partial
# “complementary distribution,” which is evidence that they are
# not distinct phonemes in the language. Thus, we could conceivably
# drop s from the Rotokas alphabet and simply have a pronunciation
# rule that the letter t is pronounced s when followed by i.

### Finding Word Stems

In [31]:
re.findall(r'^.*(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

['ing']

In [32]:
def stem(word):
    for suffix in ['ing', 'ly', 'ed', 'ious', 'ies', 'ive', 'es', 's', 'ment']:
        if word.endswith(suffix): return word[:-len(suffix)]
    return word

In [33]:
stem('potatoes')

'potato'

In [34]:
re.findall(r'^(.*)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processing')

[('process', 'ing')]

In [35]:
re.findall(r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)$', 'processes')

[('process', 'es')]

In [36]:
def stem2(word):
    regexp = r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$'
    stem, suffix = re.findall(regexp, word)[0]
    return stem

In [40]:
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
is no basis for a system of government. Supreme executive power derives from
a mandate from the masses, not from some farcical aquatic ceremony."""
tokens = nltk.word_tokenize(raw)
' '.join([stem2(t) for t in tokens])

'DENNIS : Listen , strange women ly in pond distribut sword i no basi for a system of govern . Supreme execut power deriv from a mandate from the mass , not from some farcical aquatic ceremony .'

### Searching Tokenized Text

In [42]:
moby = nltk.Text(nltk.corpus.gutenberg.words('melville-moby_dick.txt'))
moby.findall(r"<a> (<.*>) <man>")

monied; nervous; dangerous; white; white; white; pious; queer; good;
mature; white; Cape; great; wise; wise; butterless; white; fiendish;
pale; furious; better; certain; complete; dismasted; younger; brave;
brave; brave; brave


In [43]:
chat = nltk.Text(nltk.corpus.nps_chat.words())
chat.findall(r"<.*> <.*> <bro>")

you rule bro; telling you bro; u twizted bro


In [44]:
chat.findall(r"<l.*>{3,}")

lol lol lol; lmao lol lol; lol lol lol; la la la la la; la la la; la
la la; lovely lol lol love; lol lol lol.; la la la; la la la


In [45]:
hobbies_learned = nltk.Text(nltk.corpus.brown.words(categories=['hobbies', 'learned']))
hobbies_learned.findall(r"<\w*> <and> <other> <\w*s>")

speed and other activities; water and other liquids; tomb and other
landmarks; Statues and other monuments; pearls and other jewels;
charts and other items; roads and other features; figures and other
objects; military and other areas; demands and other factors;
abstracts and other compilations; iron and other metals


## Normalizing Text

In [54]:
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
is no basis for a system of government. Supreme executive power derives from
a mandate from the masses, not from some farcical aquatic ceremony."""
tokens = nltk.word_tokenize(raw)
' '.join(tokens)

'DENNIS : Listen , strange women lying in ponds distributing swords is no basis for a system of government . Supreme executive power derives from a mandate from the masses , not from some farcical aquatic ceremony .'

### Stemmers

In [56]:
porter = nltk.PorterStemmer()
' '.join([porter.stem(t) for t in tokens])

u'denni : listen , strang women lie in pond distribut sword is no basi for a system of govern . suprem execut power deriv from a mandat from the mass , not from some farcic aquat ceremoni .'

In [57]:
lancaster = nltk.LancasterStemmer()
' '.join([lancaster.stem(t) for t in tokens])

'den : list , strange wom lying in pond distribut sword is no bas for a system of govern . suprem execut pow der from a mand from the mass , not from som farc aqu ceremony .'

### Lemmatization

In [59]:
wnl = nltk.WordNetLemmatizer()
' '.join([wnl.lemmatize(t) for t in tokens])

u'DENNIS : Listen , strange woman lying in pond distributing sword is no basis for a system of government . Supreme executive power derives from a mandate from the mass , not from some farcical aquatic ceremony .'

## Regular Expressions for Tokenizing Text

### Simple Approaches to Tokenization

In [60]:
# ...

### NLTK's Regular Expression Tokenizer

In [114]:
text = 'That U.S.A. poster-print costs $12.40...'
pattern = r'''[\d.]+|[A-Z][.A-Z]+\b\.*|\w+|\S'''
nltk.regexp_tokenize(text, pattern)

['That', 'U.S.A.', 'poster', '-', 'print', 'costs', '$', '12.40...']

### Further Issues with Tokenization

In [115]:
# ...

## Segmentation

### Sentence Segmentation

In [117]:
1.0 * len(nltk.corpus.brown.words()) / len(nltk.corpus.brown.sents())

20.250994070456922

In [122]:
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
text = nltk.corpus.gutenberg.raw('chesterton-thursday.txt')
sents = sent_tokenizer.tokenize(text)
for s in sents[171:181]:
    print s
    print

In the wild events which were to follow this girl had no
part at all; he never saw her again until all his tale was over.

And yet, in some indescribable way, she kept recurring like a
motive in music through all his mad adventures afterwards, and the
glory of her strange hair ran like a red thread through those dark
and ill-drawn tapestries of the night.

For what followed was so
improbable, that it might well have been a dream.

When Syme went out into the starlit street, he found it for the
moment empty.

Then he realised (in some odd way) that the silence
was rather a living silence than a dead one.

Directly outside the
door stood a street lamp, whose gleam gilded the leaves of the tree
that bent out over the fence behind him.

About a foot from the
lamp-post stood a figure almost as rigid and motionless as the
lamp-post itself.

The tall hat and long frock coat were black; the
face, in an abrupt shadow, was almost as dark.

Only a fringe of
fiery hair against the light, and also 

### Word Segmentation

In [124]:
def segment(text, segs):
    words = []
    last = 0
    for i in range(len(segs)):
        if segs[i] == '1':
            words.append(text[last:i+1])
            last = i+1
    words.append(text[last:])
    return words

In [125]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"

In [126]:
segment(text, seg1)

['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']

In [128]:
segment(text, seg2)[:10]

['do', 'you', 'see', 'the', 'kitty', 'see', 'the', 'doggy', 'do', 'you']

In [129]:
def evaluate(text, segs):
    words = segment(text, segs)
    text_size = len(words)
    lexicon_size = len(' '.join(list(set(words))))
    return text_size + lexicon_size

In [130]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"
seg3 = "0000100100000011001000000110000100010000001100010000001"

In [132]:
segment(text, seg3)[:10]

['doyou',
 'see',
 'thekitt',
 'y',
 'see',
 'thedogg',
 'y',
 'doyou',
 'like',
 'thekitt']

In [133]:
evaluate(text, seg3)

46

In [134]:
evaluate(text, seg2)

47

In [135]:
evaluate(text, seg1)

63

In [144]:
# Non-deterministic search using simulated annealing: Begin searching with
# phrase segmentations only; randomly perturb the zeros and ones proportional
# to the “temperature”; with each iteration the temperature is lowered and
# the perturbation of boundaries is reduced.

def flip(segs, pos):
    return segs[:pos] + str(1-int(segs[pos])) + segs[pos+1:]

def flip_n(segs, n):
    for i in range(n):
        segs = flip(segs, random.randint(0,len(segs)-1))
    return segs

def anneal(text, segs, iterations, cooling_rate):
    temperature = float(len(segs))
    while temperature > 0.5:
        best_segs, best = segs, evaluate(text, segs)
        for i in range(iterations):
            guess = flip_n(segs, int(round(temperature)))
            score = evaluate(text, guess)
            if score < best:
                best, best_segs = score, guess
                score, segs = best, best_segs
                print evaluate(text, segs), segment(text, segs)
        temperature = temperature / cooling_rate
    print
    print evaluate(text, segs), segment(text, segs)
    return segs

text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
anneal(text, seg1, 5000, 1.2)

62 ['doyouseethe', 'k', 'itt', 'yseet', 'hedoggy', 'doyoul', 'iket', 'hekittyl', 'iket', 'hedoggy']
61 ['doyouseethe', 'ki', 'ttyseet', 'hedoggy', 'doyouliket', 'hekittyliket', 'hedoggy']
59 ['doyouseeth', 'eki', 'ttyseet', 'hedoggy', 'doyoulikethekittyliket', 'hedoggy']
57 ['doyouseethekitty', 'seet', 'hedoggy', 'doyoulikethekittyliket', 'hedoggy']
55 ['doyou', 'see', 'thekitty', 'see', 't', 'hedoggy', 'doyou', 'likethekittyliket', 'hedoggy']
54 ['doyou', 'see', 'thek', 'itty', 'see', 'thedoggy', 'doyou', 'likethekittylike', 'thedoggy']
53 ['doyou', 'see', 'thek', 'itty', 'see', 'thedoggy', 'doyou', 'likethek', 'itty', 'like', 'thedoggy']
52 ['doyou', 'see', 'thek', 'ittysee', 'thedoggy', 'doyou', 'like', 'thek', 'itty', 'like', 'thedoggy']
45 ['doyou', 'see', 'thek', 'itty', 'see', 'thedoggy', 'doyou', 'like', 'thek', 'itty', 'like', 'thedoggy']
42 ['doyou', 'see', 'thekitty', 'see', 'thedoggy', 'doyou', 'like', 'thekitty', 'like', 'thedoggy']

42 ['doyou', 'see', 'thekitty', 'see', 

'0000100100000001001000000010000100010000000100010000000'

## Exercises